## 准备数据

In [26]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize 像数值归一化至 0-1
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [27]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [28]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''  
        ####################

        #定义三层全连接神经网络。输入层784个神经单元，中间层256和28个神经单元，输出层10个神经单元
        self.W1 = tf.Variable(tf.random.normal([784, 256]), trainable=True)
        self.b1 = tf.Variable(tf.zeros([256]), trainable=True)
        self.W2 = tf.Variable(tf.random.normal([256, 128]), trainable=True)
        self.b2 = tf.Variable(tf.zeros([128]), trainable=True)
        self.W3 = tf.Variable(tf.random.normal([128, 10]), trainable=True)
        self.b3 = tf.Variable(tf.zeros([10]), trainable=True)

    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################

        #前向传播
        x = tf.reshape(x, [-1, 28*28])
        h1 = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
        h2 = tf.nn.relu(tf.matmul(h1, self.W2) + self.b2)
        logits = tf.matmul(h2, self.W3) + self.b3
        return logits

        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [29]:
@tf.function
def compute_loss(logits, labels):  #使用交叉熵损失函数计算模型输出与真实标签之间的损失
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):  #使用梯度下降法进行模型训练，通过反向传播更新模型参数，以最小化损失函数
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [30]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 1701.5756 ; accuracy 0.091133334
epoch 1 : loss 1115.8579 ; accuracy 0.1372
epoch 2 : loss 720.1622 ; accuracy 0.21125
epoch 3 : loss 497.8126 ; accuracy 0.25991666
epoch 4 : loss 342.81784 ; accuracy 0.35481668
epoch 5 : loss 294.48416 ; accuracy 0.40223333
epoch 6 : loss 263.5202 ; accuracy 0.42975
epoch 7 : loss 240.94737 ; accuracy 0.46261665
epoch 8 : loss 222.76605 ; accuracy 0.48168334
epoch 9 : loss 208.01857 ; accuracy 0.5043667
epoch 10 : loss 195.63336 ; accuracy 0.5197333
epoch 11 : loss 185.03758 ; accuracy 0.53618336
epoch 12 : loss 175.8219 ; accuracy 0.5492167
epoch 13 : loss 167.68242 ; accuracy 0.5623
epoch 14 : loss 160.46512 ; accuracy 0.5732333
epoch 15 : loss 153.97603 ; accuracy 0.58318335
epoch 16 : loss 148.10664 ; accuracy 0.59325
epoch 17 : loss 142.79103 ; accuracy 0.6023833
epoch 18 : loss 137.94492 ; accuracy 0.61083335
epoch 19 : loss 133.49722 ; accuracy 0.6188833
epoch 20 : loss 129.41156 ; accuracy 0.6267167
epoch 21 : loss 125.64906 ; a